In [1]:
import ee
from scipy.signal import savgol_filter
from src.controllers.time_series_hls import HLS
from src.controllers.plotter_base import PhenologyPlotter
from src.controllers.metrics_vos_pos import VosPosMetrics
from src.controllers.metrics_bos_eso import BosEosMetrics
from src.controllers.geometry import ProcessadorGeoDataFrame
from src.controllers.metrics_geometrics import PhenologyMetrics


In [2]:
# Conecta ao Google Earth Engine
ee.Authenticate()
ee.Initialize()

EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [ ]:
# Datas de análise
start_date = '2022-12-01'
end_date = '2024-01-31'

#Paht polygons
path = r"data\farms.geojson"

#Index polygon, defoult is 0
index_poligon = 2

# Aplicar o filtro Savitzky-Golay
window_size = 30
poly_order = 3

#Order NDVI
order_ndvi = 30

In [ ]:
#Read geometry file
processador = ProcessadorGeoDataFrame(path)

for index_poligon in range(400): 
    try:

        vertices, geometry = processador.extrair_coordenadas(index_poligon)

        #Get the time sereis NDVI
        hsl = HLS(geometry, start_date, end_date)
        ndvi_df = hsl.convert_to_dataframe()

        #Smooth time series with NDVI
        ndvi_df['savitzky_golay'] = savgol_filter(ndvi_df['ndvi'], window_size, poly_order)

        #Get and VOS and POS metrics
        vos_pos_analyzer = VosPosMetrics(ndvi_df, order_ndvi)
        phenology_df = vos_pos_analyzer.analyze_phenology()

        #Get and BOS and EOS metrics
        analysis = BosEosMetrics(ndvi_df, phenology_df, 0.3)
        phenology_df  = analysis.execute_analysis()

        analysis_metrics = PhenologyMetrics(phenology_df, ndvi_df)
        phenology_df  = analysis_metrics.derivate_metrics()

        plotter = PhenologyPlotter(ndvi_df, phenology_df)
        fig = plotter.plot_data()

        fig.write_html(rf"htmls\{index_poligon}.html")

    
    except:
        pass


c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\time_series_hls.py:125: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('D').interpolate(method='linear')
c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.interval_df['first_derivative'] = self.interval_df['savitzky_golay'].diff().fillna(0)
c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\time_series_hls.py:125: FutureWarning:

DataFrame.interpolate with object dtype is deprecated and will rai

In [ ]:
#Read geometry file
processador = ProcessadorGeoDataFrame(path)
vertices, geometry = processador.extrair_coordenadas(index_poligon)

#Get the time sereis NDVI
hsl = HLS(geometry, start_date, end_date)
ndvi_df = hsl.convert_to_dataframe()

#Smooth time series with NDVI
ndvi_df['savitzky_golay'] = savgol_filter(ndvi_df['ndvi'], window_size, poly_order)

#Get and VOS and POS metrics
vos_pos_analyzer = VosPosMetrics(ndvi_df, order_ndvi)
phenology_df = vos_pos_analyzer.analyze_phenology()

#Get and BOS and EOS metrics
analysis = BosEosMetrics(ndvi_df, phenology_df, 0.3)
phenology_df  = analysis.execute_analysis()

analysis_metrics = PhenologyMetrics(phenology_df, ndvi_df)
phenology_df  = analysis_metrics.derivate_metrics()

plotter = PhenologyPlotter(ndvi_df, phenology_df)
fig = plotter.plot_data()

fig.show()

c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\time_series_hls.py:125: FutureWarning:

DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.

c:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig.write_html(r"C:\Users\ander\OneDrive\Área de Trabalho\msu\msu_timeseries_ndvi\2.html")

fig.show()

In [ ]:
import pandas as pd
from datetime import timedelta

class PhenologyMetrics:
    def __init__(self, phenology_df, ndvi_df ):
        self.phenology_df = phenology_df
        self.ndvi_df = ndvi_df

    def days_between(self, start_event, end_event):
        """Calculate the number of days between two phenological events."""
        start_date = self.phenology_df.loc[self.phenology_df['Phenologic'] == start_event, 'Date'].values[0]
        end_date = self.phenology_df.loc[self.phenology_df['Phenologic'] == end_event, 'Date'].values[0]
        return (end_date - start_date ).astype('timedelta64[D]').astype(int)

    def vertical_difference(self, start_event, end_event):
        """Calculate the vertical difference (in NDVI value) between two events."""
        start_value = self.phenology_df.loc[self.phenology_df['Phenologic'] == start_event, 'Value'].values[0]
        end_value = self.phenology_df.loc[self.phenology_df['Phenologic'] == end_event, 'Value'].values[0]
        return end_value - start_value

    def horizontal_difference(self, start_event, end_event):
        """Calculate the horizontal difference (in days) between two events."""
        return self.days_between(start_event, end_event)
    
    def percentil_difference(self):
        start_date = self.phenology_df.loc[self.phenology_df['Phenologic'] == 'bos_abs', 'Date'].values[0]
        end_date = self.phenology_df.loc[self.phenology_df['Phenologic'] == 'eos_abs', 'Date'].values[0]

        # Filtrar as linhas entre as datas especificadas
        mask = (self.ndvi_df['date'] >= pd.to_datetime(start_date)) & (self.ndvi_df['date'] <= pd.to_datetime(end_date))
        filtered_data = ndvi_df.loc[mask]

        # Calcular o valor do percentil na coluna NDVI
        percentile_value = filtered_data['savitzky_golay'].quantile(85 / 100)

        # Contar quantos valores de NDVI estão acima do valor do percentil
        return (filtered_data['savitzky_golay'] > percentile_value).sum()


# Criando uma instância da classe com o DataFrame
phenology_metrics = PhenologyMetrics(phenology_df, ndvi_df)

# Testes das métricas
print("Days between vos_end and vos_start:", phenology_metrics.days_between('vos_start', 'vos_end')) #1 
print("Days between bos_abs and eos_abs:", phenology_metrics.days_between('bos_abs', 'eos_abs')) #2
print("Vertical difference between bos_abs and pos:", phenology_metrics.vertical_difference('bos_abs', 'pos'))#3
print("Horizontal difference between bos_abs and pos:", phenology_metrics.horizontal_difference('bos_abs', 'pos'))#4
print("Horizontal difference between vos_start and bos_abs:", phenology_metrics.horizontal_difference('vos_start', 'bos_abs'))#5
print("Vertical difference between vos_start and bos_abs:", phenology_metrics.vertical_difference('vos_start', 'bos_abs'))#6
print("Horizontal difference between vos_end and eos_abs:", phenology_metrics.horizontal_difference('eos_abs', 'vos_end'))#7
print("Vertical difference between eos_abs and pos:", phenology_metrics.vertical_difference('vos_start', 'bos_abs'))#8
print("Horizontal difference between eos_abs and pos:", phenology_metrics.horizontal_difference('pos', 'eos_abs'))#9
print("Vertical difference between eos_abs and pos:", phenology_metrics.vertical_difference('eos_abs', 'pos'))#10


Days between vos_end and vos_start: 189
Days between bos_abs and eos_abs: -146
Vertical difference between bos_abs and pos: 0.3182379948822235
Horizontal difference between bos_abs and pos: -85
Horizontal difference between vos_start and bos_abs: 171
Vertical difference between vos_start and bos_abs: 0.03754055342438245
Horizontal difference between vos_end and eos_abs: 164
Vertical difference between eos_abs and pos: 0.03754055342438245
Horizontal difference between eos_abs and pos: -61
Vertical difference between eos_abs and pos: 0.3029829148067714
